In [2]:
import torch
import os
import mmcv
import sys
import cv2
import copy
import numpy as np
import argparse
import subprocess
import math

root_dir = '/n/fs/vai-bwc-scr/'

TW_dir     = os.path.join(root_dir, 'annotations/TW_annotations')
TW_paths   = [os.path.join(TW_dir, x) for x in os.listdir(TW_dir)]
title_path = os.path.join(root_dir, 'annotations/URLs/ids_and_titles.txt')

out_dir = os.path.join(root_dir, 'TW_videos')


In [3]:
IDs_to_titles = {}
with open(title_path, 'r') as f:
    for line in f:
        line = line.strip()
        ID, title = line[:9], line[10:]
        title = title.replace(' ', '*')
        title = ''.join(title.split())
        title = title.replace('*', ' ') + '.mp4'
        IDs_to_titles[ID] = title
TWs = {}
for TW_path in TW_paths:
    if not "BWC" in TW_path or '.swp' in TW_path:
        continue
    with open(TW_path, 'r') as f:
        for line in f:
            if 'vimeo' in line: 
                curr_ID = line.strip().split()[0][-9:]
                TWs[curr_ID] = []
            elif line.strip() == '':
                continue
            else:
                line_split = line.strip().split()
                t1, t2 = line_split[:2]
                timeframe = []
                for t in [t1, t2]:
                    minute, second = int(t[:2]), int(t[-2:])
                    timeframe.append(minute*60 + second)
                TWs[curr_ID].append(timeframe)

for vid_id in TWs:
    title = IDs_to_titles[vid_id]
    vid_path = os.path.join(out_dir, title)
    if os.path.exists(vid_path):
            print('\tVideo already processed. Skipping.')
            continue
    subprocess.call(['youtube-dl', 'https://vimeo.com/'+str(vid_id), '-o', out_dir+'/%(title)s.%(ext)s'])